## Data and Training

The **augmented** cough audio dataset of the [Project Coswara](https://coswara.iisc.ac.in/about) was used to train the deep CNN model.

The preprocessing steps and CNN architecture is as shown below. The training code is concealed to protect the exact hyperparameters and maintain performance integrity of the model.
<img src = "../assets/ml-pipeline.png" alt="ML pipeline" width="800"/>


## Model Deployment on IBM Watson Machine Learning

Below are the contents of an IBM Watson Studio Notebook for deploying our trained ML model IBM Watson Machine Learning.

Outputs, Keys, Endpoints and URLs are removed (replaced with <>) to maintain privacy.

### Import model

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_2 = {
    'IAM_SERVICE_ID': <>,
    'IBM_API_KEY_ID': <>,
    'ENDPOINT': <>,
    'IBM_AUTH_ENDPOINT': <>,
    'BUCKET': <>,
    'FILE': 'cough-it-model.tgz'
}


In [ ]:
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_2['IBM_API_KEY_ID'], 
    ibm_auth_endpoint=credentials_2['IBM_AUTH_ENDPOINT'],
    ibm_service_instance_id=credentials_2['IAM_SERVICE_ID'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_2['ENDPOINT'])

cos.download_file(Bucket=credentials_2['BUCKET'], Key='cough-it-model.h5.tgz', Filename='cough-it-model.h5.tgz')
model_path = 'cough-it-model.h5.tgz'


### Set up Watson Machine Learning Client and Deployment space

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "apikey"    : <>,
    "url"    : <>
}

client = APIClient( wml_credentials )

In [ ]:
space_guid = <>
client.set.default_space(space_guid)

### Store the model

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.8")

metadata = {
    client.repository.ModelMetaNames.NAME: "cough-it model",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.4"
}
published_model = client.repository.store_model( model= model_path, meta_props=metadata )

In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

### Create a deployment

In [ ]:
dep_metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Keras model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=dep_metadata)


In [ ]:
deployment_uid = client.deployments.get_uid(created_deployment)
client.deployments.get_details(deployment_uid)